In [1]:
import tensorflow as tf
import numpy as np
from time import time

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
training_mode = input("Training mode (Y/F):")
training_mode = training_mode == "Y"
print("Training mode:", training_mode)

Training mode (Y/F):Y
Training mode: True


In [3]:
epochs = 1
batch_size = 32
ckpt_dir = "/tmp/tensor_cp/"

Load data

In [4]:
def load_mnist_csv(path = "/data/MNIST/", one_hot = False, shape = None):
    import pandas as pd
    df_train = pd.read_csv(path + "mnist_train.csv", header=None)
    df_test = pd.read_csv(path + "mnist_test.csv", header=None)
    
    X_train = df_train.iloc[:, 1:].values/255
    X_test = df_test.iloc[:, 1:].values/255
    y_train = df_train.iloc[:, 0].values
    y_test = df_test.iloc[:, 0].values
    
    if shape == "3D":
        X_train = X_train.reshape(-1, 28, 28, 1)
        X_test = X_test.reshape(-1, 28, 28, 1)
    
    if one_hot:
        eye = np.eye(10)
        y_train, y_test = eye[y_train], eye[y_test]
        
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_mnist_csv(one_hot=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 784), (10000, 784), (60000, 10), (10000, 10))

Build the data generator

In [5]:
def data_generator(X, y, batch_size = 32, epochs = 1):
    from collections import namedtuple
    from math import ceil
    Batch = namedtuple("batch", ["epoch", "global_step", "progress", "X_batch", "y_batch"])
    global_step = 0
    for epoch in range(epochs):
        m = X_train.shape[0]
        indices = np.arange(m)
        np.random.shuffle(indices)
        X = X[indices]
        y = y[indices]
        num_batches = ceil(m/batch_size)
        for j in range(num_batches):
            start = j * batch_size
            end = start + batch_size
            X_batch = X[start:end]
            y_batch = y[start:end]
            progress = (j + 1) * 100 / num_batches
            yield Batch(epoch, global_step, progress, X_batch, y_batch)
            global_step = global_step + 1

Initialize the computation graph

In [6]:
tf.reset_default_graph()
graph = tf.get_default_graph()

Input to the computation graph

In [7]:
X = tf.placeholder(dtype=tf.float32, shape=(None, 784))
y = tf.placeholder(dtype=tf.int32, shape=(None,10))
dropout_rate = tf.placeholder_with_default(0.0, shape=())  

Build computation graph

In [8]:
X_img = tf.reshape(X, shape=(-1, 28, 28, 1))
    
layers = [
    tf.layers.Conv2D(32, (5, 5), activation=tf.nn.relu),
    tf.layers.MaxPooling2D((2, 2), (2, 2)),
    tf.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
    tf.layers.MaxPooling2D((2, 2), (2, 2)),
    tf.layers.Flatten(),
    tf.layers.Dropout(dropout_rate),
    tf.layers.Dense(400, tf.nn.relu),
    tf.layers.Dense(100, tf.nn.relu),
    tf.layers.Dense(10, None)
]
logits = X_img
for layer in layers:
    logits = layer(logits)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
cost = tf.reduce_mean(loss)

tf.summary.scalar("cost", cost)

op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

macthes = tf.equal(tf.argmax(y, axis=1), tf.argmax(logits, axis=1))

accuracy = tf.reduce_mean(tf.cast(macthes, tf.float32))

Create a tensor saver object to save or restore the variables

In [9]:
saver = tf.train.Saver()

Show all variables

In [10]:
graph.get_collection("variables")

[<tf.Variable 'conv2d/kernel:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(1600, 400) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(400, 100) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(100, 10) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel/Adam:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel/Adam_1:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias/Adam:0' shape=(32,) dtype=flo

Create summary for the accuracy, input images and the kernel to show them in the tensorboard 

In [11]:
tf.summary.scalar("accuracy", accuracy)
tf.summary.image("input_images", X_img)
for v in tf.global_variables():
    if "kernel" in v.name:
        tf.summary.histogram(v.name.replace(":", "_"), v)
summary_agg = tf.summary.merge_all()

Run training, write the summary to tensorboard and save the model

In [12]:
if training_mode:
    tf.set_random_seed(1)
    np.random.seed(1)
    
    #Create a file write to write summary and graph to tensorboard
    writer = tf.summary.FileWriter(logdir="/tmp/tensorboard/%d" % time(), graph=tf.get_default_graph())
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for batch in data_generator(X_train, y_train, batch_size, epochs):
            _, accuracy_, cost_, summary_ = sess.run([op, accuracy, cost, summary_agg]
                                , feed_dict={X: batch.X_batch, y: batch.y_batch, dropout_rate: 0.8})
            writer.add_summary(summary_, batch.global_step)
            print("Epoch: %d, progress: %3d, cost: %.4f, acc: %.4f" 
                  % (batch.epoch, batch.progress, cost_, accuracy_), end="\r")
            
            if batch.progress == 100:
                acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
                print("\n validation accuracy: %.4f" % acc)

        saver.save(sess, ckpt_dir)
        print(sess.run("conv2d/kernel:0")[0][0])
else: 
    print("[Inferencing mode] No training is performed")

Epoch: 0, progress: 100, cost: 0.0115, acc: 1.0000
 validation accuracy: 0.9865
[[-0.04012164 -0.10293753  0.01783721 -0.06238898  0.08442013 -0.06689017
  -0.03600748  0.10734922 -0.12466065  0.04650401 -0.03229889  0.13118029
  -0.01973842  0.06481586  0.09878534 -0.02696361  0.04368662 -0.07343274
   0.08764903  0.02837615  0.00431452 -0.01515117  0.01117284  0.10848288
  -0.06746104  0.0613336   0.07979139  0.16718565  0.0205169  -0.00472976
  -0.12347219  0.07548217]]


In [14]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, ckpt_dir)
    acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
    print("\nvalidation accuracy: %.4f" % acc)
    print("\nconv2d/kernel:0\n", sess.run("conv2d/kernel:0")[0][0])

INFO:tensorflow:Restoring parameters from /tmp/tensor_cp/

validation accuracy: 0.9865

conv2d/kernel:0
 [[-0.04012164 -0.10293753  0.01783721 -0.06238898  0.08442013 -0.06689017
  -0.03600748  0.10734922 -0.12466065  0.04650401 -0.03229889  0.13118029
  -0.01973842  0.06481586  0.09878534 -0.02696361  0.04368662 -0.07343274
   0.08764903  0.02837615  0.00431452 -0.01515117  0.01117284  0.10848288
  -0.06746104  0.0613336   0.07979139  0.16718565  0.0205169  -0.00472976
  -0.12347219  0.07548217]]
